In [379]:
%pylab inline
import numpy as np
from scipy import special
import pandas as pd
import os

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [380]:
METHOD_NAME = "nn_part-32_dropout0.5_40-3"
DATA_FOLDER = "data/"
MODEL_FOLDER = "models/" + METHOD_NAME + "/"
OUTPUT_FOLDER = "outputs/"
TRAIN_FILE_PATH = DATA_FOLDER + "spam_train.csv"
TEST_FILE_PATH = DATA_FOLDER + "spam_test.csv"

In [381]:
if not os.path.exists(MODEL_FOLDER):
    os.makedirs(MODEL_FOLDER)
if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)

## Read Training Data

In [382]:
train_data = pd.read_csv(TRAIN_FILE_PATH, header=None)
train_data.head()

,0,1,2,3,4,5,6,7,8,9,...,49,50,51,52,53,54,55,56,57,58
0,1,0.0,0.00,1.13,0.0,0.37,0.00,0.00,0.0,0.00,...,0.0,0.145,0.000,0.436,0.000,0.0,1.792,55,147,0
1,2,0.0,0.00,0.60,0.0,0.00,0.60,0.00,0.0,0.60,...,0.0,0.143,0.047,0.191,0.143,0.0,2.041,31,196,1
2,3,0.0,0.00,0.48,0.0,0.00,0.00,0.00,0.0,0.00,...,0.0,0.000,0.000,0.450,0.000,0.0,1.138,4,41,0
3,4,0.0,0.51,0.00,0.0,0.51,0.51,1.02,0.0,0.00,...,0.0,0.142,0.000,0.071,1.212,0.0,7.025,130,281,1
4,5,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.64,...,0.0,0.116,0.000,0.232,0.000,0.0,1.551,6,45,0


In [383]:
x_data_rows = train_data[list(range(1, 58))].as_matrix()
y_data_rows = train_data[[58]].as_matrix()

## Select Data

In [384]:
x_select_col_idxs = [26, 40, 31, 24,  3, 41, 54, 55, 45, 52, 28, 22, 53, 34, 43, 47, 6, 44, 15, 25, 16, 19, 38,  8, 56, 32,  4, 35, 27,  5, 48, 14]
x_data_rows = x_data_rows[:, x_select_col_idxs]

## Preprocess Data

## Transform Data

In [385]:
# Normalize
x_train_means = np.mean(x_data_rows, axis=0)
x_train_stds =  np.std(x_data_rows, axis=0)
x_data_rows = (x_data_rows - x_train_means) / x_train_stds

In [386]:
# print(np.mean(x_data_rows, axis=0), np.std(x_data_rows, axis=0))

In [387]:
# append 1 in the last column, so that b in including in w
x_data_rows = np.c_[x_data_rows, np.ones(x_data_rows.shape[0]) ]

In [388]:
# pre-transpose x for matrix multiplication
x_data_rows_trans = np.transpose(x_data_rows)

In [389]:
x_row_num, x_col_num = x_data_rows.shape  # 4001, 58

## Train

In [390]:
# reset parameters
w = np.random.randn(x_col_num, 1)  # shape = (x_col_num, 1). not (1, x_col_num) for the convenience of matrix multiplication
total_epoch = 0

In [391]:
layer_1_node_num = 40
layer_2_node_num = layer_1_node_num
layer_3_node_num = layer_2_node_num

In [392]:
w_1 = np.random.randn(x_col_num, layer_1_node_num)  # dot(x, w_1) output a x_row_num * layer_1_node_num matrix
w_2 = np.random.randn(layer_1_node_num, layer_2_node_num)
w_3 = np.random.randn(layer_2_node_num, layer_3_node_num)
w_out = np.random.randn(layer_3_node_num, 1)  # dot(layer_1_output, w_2) output a x_row_num * 1 matrix
# v_w_1 = np.random.randn(x_col_num, layer_1_node_num)  # dot(x, w_1) output a x_row_num * layer_1_node_num matrix
# v_w_2 = np.random.randn(layer_1_node_num, 1)  # dot(layer_1_output, w_2) output a x_row_num * 1 matrix

In [393]:
dropout_rate = 0.5

In [394]:
# momentum
gamma = 0.9
# vw = np.zeros((x_col_num, 1))

In [395]:
lr = (10) / x_row_num 

In [396]:
def to_bool(x):
    return 1 if x > 0.5 else 0

to_bool = np.vectorize(to_bool)

In [397]:
def sigmoid_deriv(x):  # input may be an matrix
    return x * (1-x)  # element-wise

In [398]:
# a = np.ones((3, 10))
# m = np.random.binomial(1, 0.5, 10)
# a * m

In [410]:
# training iterations

# special.expit is sigmoid

layer_0_output = x_data_rows

while True:
    layer_1_output = special.expit(np.dot(layer_0_output, w_1))  # shape = (x_row_num, layer_1_node_num)
    mask_1 = numpy.random.binomial(1, dropout_rate, layer_1_node_num)  # TODO: check if need *2
    layer_1_output *= mask_1

    layer_2_output = special.expit(np.dot(layer_1_output, w_2))  # shape = (x_row_num, layer_2_node_num)
    mask_2 = numpy.random.binomial(1, dropout_rate, layer_2_node_num)
    layer_2_output *= mask_2

    layer_3_output = special.expit(np.dot(layer_2_output, w_3))  # shape = (x_row_num, layer_3_node_num)
    mask_3 = numpy.random.binomial(1, dropout_rate, layer_3_node_num)
    layer_3_output *= mask_3

    layer_out_output = special.expit(np.dot(layer_3_output, w_out))  # shape = (x_row_num, 1)

    # backpropagation
    layer_out_error = layer_out_output - y_data_rows
    layer_out_delta = layer_out_error * sigmoid_deriv(layer_out_output)

    layer_3_error = layer_out_delta.dot(w_out.T)
    layer_3_delta = layer_3_error * sigmoid_deriv(layer_3_output)

    layer_2_error = layer_3_delta.dot(w_3.T)  # shape = (x_row_num, layer_1_node_num)
    layer_2_delta = layer_2_error * sigmoid_deriv(layer_2_output)  # shape = (x_row_num, layer_1_node_num)

    layer_1_error = layer_2_delta.dot(w_2.T)  # shape = (x_row_num, layer_1_node_num)
    layer_1_delta = layer_1_error * sigmoid_deriv(layer_1_output)  # shape = (x_row_num, layer_1_node_num)

    w_out -= lr * (layer_3_output.T).dot(layer_out_delta)  # (20, 4001) * (4001, 1)
    w_3 -= lr * (layer_2_output.T).dot(layer_3_delta)  # shape = (layer_1_node_num, 1)
    w_2 -= lr * (layer_1_output.T).dot(layer_2_delta)  # shape = (layer_1_node_num, 1)
    w_1 -= lr * (layer_0_output.T).dot(layer_1_delta)  # shape = (x_row_num, layer_1_node_num)

#     v_w_2 = lr * (layer_1_output.T).dot(layer_2_delta) + gamma * v_w_2
#     v_w_1 = lr * (layer_0_output.T).dot(layer_1_delta) + gamma * v_w_1
#     w_2 += v_w_2  # shape = (layer_1_node_num, 1)
#     w_1 += v_w_1  # shape = (x_row_num, layer_1_node_num)

    # write model
    if (total_epoch < 10000 and total_epoch % 1000 == 0) or (total_epoch % 10000 == 0):
        layer_1_output = dropout_rate * special.expit(np.dot(layer_0_output, w_1))
        layer_2_output = dropout_rate * special.expit(np.dot(layer_1_output, w_2))
        layer_3_output = dropout_rate * special.expit(np.dot(layer_2_output, w_3))
        layer_out_output = special.expit(np.dot(layer_3_output, w_out))
        layer_out_error = layer_out_output - y_data_rows

        pred_error = np.sum(np.abs(to_bool(layer_out_output) - y_data_rows))
        rms_error = np.mean(np.square(layer_out_error))

        model_file_info = "epo" + str(total_epoch) + "_rms" + str(rms_error)[:8] + "_err" + str(pred_error)
        print('====', model_file_info)
        print('rms_error', rms_error)
        print('pred_error', pred_error, '/', x_row_num, '=', pred_error/x_row_num)

        np.savez(MODEL_FOLDER + model_file_info, w_1=w_1, w_2=w_2, w_3=w_3, w_out=w_out, lr=lr, dropout_rate=dropout_rate, total_epoch=total_epoch, x_train_means=x_train_means, x_train_stds=x_train_stds)

#     vw = lr * w_grad + gamma * vw
#     w = w - vw

    total_epoch += 1


==== epo1100000_rms0.028484_err138
rms_error 0.0284843361773
pred_error 138 / 4001 = 0.0344913771557
==== epo1110000_rms0.028358_err139
rms_error 0.0283581469027
pred_error 139 / 4001 = 0.0347413146713
==== epo1120000_rms0.028369_err139
rms_error 0.0283693399288
pred_error 139 / 4001 = 0.0347413146713
==== epo1130000_rms0.028307_err138
rms_error 0.0283072961218
pred_error 138 / 4001 = 0.0344913771557
==== epo1140000_rms0.028328_err138
rms_error 0.0283280791041
pred_error 138 / 4001 = 0.0344913771557
==== epo1150000_rms0.028257_err137
rms_error 0.0282573301272
pred_error 137 / 4001 = 0.0342414396401
==== epo1160000_rms0.028179_err138
rms_error 0.0281796167894
pred_error 138 / 4001 = 0.0344913771557
==== epo1170000_rms0.028195_err137
rms_error 0.0281957337956
pred_error 137 / 4001 = 0.0342414396401
==== epo1180000_rms0.028317_err139
rms_error 0.0283175341376
pred_error 139 / 4001 = 0.0347413146713
==== epo1190000_rms0.028192_err139
rms_error 0.0281929516718
pred_error 139 / 4001 = 0.0347

KeyboardInterrupt: 

## Test

In [411]:
test_data = pd.read_csv(TEST_FILE_PATH, header=None)

In [412]:
_id = test_data[0].apply(str)

In [413]:
x_test_data_rows = test_data[list(range(1, 58))].as_matrix()

In [414]:
x_test_data_rows = x_test_data_rows[:, x_select_col_idxs]

In [415]:
# # read model
# model_file = 'epo900000_rms0.020180_err97.npz'
# model = np.load(MODEL_FOLDER + model_file)
# w_1 = model['w_1']
# w_2 = model['w_2']
# w_3 = model['w_3']
# w_out = model['w_out']
# model_file_info = model_file[:-4]

In [416]:
# normalize
x_test_data_rows = (x_test_data_rows - x_train_means) / x_train_stds
# add 1
x_test_data_rows = np.c_[x_test_data_rows, np.ones(x_test_data_rows.shape[0]) ]

In [417]:
# test
test_layer_1_output = dropout_rate * special.expit(np.dot(x_test_data_rows, w_1))
test_layer_2_output = dropout_rate * special.expit(np.dot(test_layer_1_output, w_2))
test_layer_3_output = dropout_rate * special.expit(np.dot(test_layer_2_output, w_3))
test_layer_out_output = special.expit(np.dot(test_layer_3_output, w_out))
y_test_rows = to_bool(test_layer_out_output)

y_test_series = pd.Series(y_test_rows.flatten())

# concat id and y
output = pd.concat([_id, y_test_series], axis=1)
output.columns=["id","label"]

# write file
output.to_csv(OUTPUT_FOLDER + "output_" + METHOD_NAME + "_" + model_file_info + ".csv", index=False)